In [36]:
import sys
import os

sys.path.append('../')

**1- Data Preparation**

In [ ]:
dataset_name = 'Enron'
#dataset_name = 'Drebin'  # we use the spam dataset as an example
normalize = True  # we normalize the datapoints to have euclidean norm of 1
# load the training and test data
loader = DataLoader(dataset_name, normalize)
train_data, test_data, voc = (loader.x_train, loader.y_train), (loader.x_test, loader.y_test), loader.voc
category_to_idx_dict = loader.category_to_idx_dict
# create folders for results
res_save_folder = 'Results_{}'.format(dataset_name)
model_save_folder = 'Models_{}'.format(dataset_name)
if not os.path.isdir(res_save_folder):
    os.makedirs(res_save_folder)
# relevant features contain the privacy sensitive features of the dataset, (names in the case of Enron)
relevant_features = loader.relevant_features
relevant_indices = [voc[f] for f in relevant_features]
# lets see some names that were found in the dataset
print(','.join(relevant_features[:10]))
# these features will be deleted
indices_to_delete = relevant_indices

**2- Model Training:**

In [44]:
# import Random classifier
from sklearn.ensemble import RandomForestClassifier
modele_A = RandomForestClassifier(n_estimators=100, random_state=42)
modele_A.fit(train_data[0], train_data[1])


RandomForestClassifier()

In [45]:
# evaluation
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score

y_pred = modele_A.predict(test_data[0])
acc = accuracy_score(test_data[1], y_pred)
f1 = f1_score(test_data[1], y_pred)
precision = precision_score(test_data[1], y_pred)
conf_matrix = confusion_matrix(test_data[1], y_pred)
print('Accuracy: ', acc)
print('F1: ', f1)
print('Precision: ', precision)
print('Confusion matrix: ')
print(conf_matrix)


Accuracy:  0.9847271648873073
F1:  0.9853672396647251
Precision:  0.9771766694843618
Confusion matrix: 
[[3173   81]
 [  22 3468]]


In [ ]:
import scipy.sparse as sp

def copy_and_replace(x, indice, remove=False, n_replacements=0):
    x_cpy = x.copy()

    if sp.issparse(x):
        x_cpy = x_cpy.tolil()
        

**3- Certified Unlearning Process:**

In [47]:
lambda_ = 0.1
sigma = 0.1 
epsilon = 0.1
indices_to_delete[:10]

[269, 318, 352, 361, 402, 424, 485, 496, 565, 616]

**4- Evaluation:**

**5- Documentation and Reporting:**

**6- Validation and Certification:**